In [23]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

import pandas as pd
import torch
import transformers
from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModelForCausalLM
from openai import OpenAI

load_dotenv("../.env")
openai_api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
eng_responses = [] # save responses in list to later set as column in dataframe
eng_data = pd.read_excel("eng_data.xlsx")

model = "gpt-4o-mini"
client = OpenAI(api_key=openai_api_key)
system_prompt = "You are a helpful assistant aiding in the generation of a Q&A dataset. When given a text, your task is to generate a prompt that the text could have been a response to."
chat_history = [{"role": "system", "content": system_prompt}]

for _, row in eng_data.iterrows():
    # iterate over rows of dataframe
    chat_history = [{"role": "system", "content": system_prompt}]
    text = row["text"]
    prompt = f"""Given the following text, please generate a prompt that it could be a response for:
                \"{text}\"
                """
    chat_history.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model=model,
        messages=chat_history
    )

    reply = response.choices[0].message.content.strip()
    eng_responses.append(reply)
eng_data["prompt"] = eng_responses
eng_data.to_excel("eng_data_pseudoprompt.xlsx", index=False)